# Law, Bias, and Algorithms
## Narrow Tailoring and Disparate Impact in Law School Admissions
This notebook will provide an example of how algoritms can be used to classify individuals. Specifically, it will consider how top tier law schools select admits. 

This notebook uses the example of law school admissions and considers a whether top law schools would be able to achieve current levels of diversity without directly using race in admissions. 

This notebook proceeds by first exploring a data set on law school admissions. It then reverse engineers a possible model for law school admissions and asks you to consdier alternative models. Finnally, the notebook ends by considering whether predicted bar passage could serve as a better metric for law school admissions. Throughout the notebook asks you to consider how you could implement an admission algorithm while maintaining diversity in top law schools. 

In [1]:
# Some initial setup
options(digits = 3)
library(tidyverse)
theme_set(theme_bw())

# Read the data
data <- read_csv("../data/bar_passage_data.csv", 
                 col_types = cols(MINORITY="l", TOP_TIER="l", MALE="l", PASS_BAR="l")) %>% 
    mutate(FAM_INC = as.factor(FAM_INC))

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.7
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Each row in the data corresponds to a law school admit. The dataset contains the following variables:

* An ID number:
    * `ID`
    
    
* Base demographic information about the applicant:
    * `MINORITY`, `MALE`
    * `RACE` is encoded as follows:        
        * `False`: Non-hispanic White
        * `True`: Asian, Black, Hispanic, American Indian, Alaskan Native, or Other
    * `MALE` is coded as `True` for male applicants and `False` for female applicants
        
        
* Outcome of interest, Bar Passage:
    * `PASS_BAR`, `BAR`
    * `PASS_BAR` is an indicator variable and is encoded as 0 regardless of why the student did not pass the exam.  They may have dropped out of law school, never taken the bar, or failed the exam. `PASS_BAR` is encoded as 1 if the student eventually passes the bar. 
    * `BAR` provides more detail about bar results and test history
    
 
* Academic Indicators:
    * `UGPA` (undergraduate GPA), `LSAT` (LSAT score, scaled to be between 10 and 50)
    
    
* Tier of Law School Attended:
    * `TOP_TIER` is an indicator variable for whether an applicant ultmiately attends a top tier school
    * Note that students who attend historically black colleges and universities were removed as those schools are outliers in law school admissions.


* Family Income Quintile:
    * `FAM_INC` provides the family income quintile
    * `FAM_INC_1`, `FAM_INC_2`, `FAM_INC_3`, `FAM_INC_4`,` FAM_INC_5` are indicator variables for the income quintile

Law school admits whose entries had missing data have been removed.

### Exploratory Data Analysis

We will start our analysis by exploring racial decomposition and disparities.

#### Excercise 1: Current Demographic Composituion and Disparities

1. Show the mean LSAT and undergraduate GPA by race
2. Create a table showing the total number of law school admits, the number of minority admits, and the percentage of law school admits who are minorties
3. Recalculate the same statistics in (2), but for top tier law schools only.

In [2]:
#WRITE CODE HERE

#BEGIN SOLUTION

# 1.
data %>% 
    group_by(MINORITY) %>%
    summarize(
        mean_LSAT = mean(LSAT),
        mean_UGPA = mean(UGPA)
    )

# 2.
# Minority distribution in all schools
data %>%
    summarize(
        total_admits = n(),
        minority_admits = sum(MINORITY),
        minority_proportion = minority_admits/total_admits
    )

# 3.
# Minority distribution in top tier schools
data %>%
    filter(TOP_TIER) %>%
    summarize(
        total_admits = n(),
        minority_admits = sum(MINORITY),
        minority_proportion = minority_admits/total_admits
    )
#END SOLUTION

MINORITY,mean_LSAT,mean_UGPA
FALSE,37.2,3.25
TRUE,32.0,3.01


total_admits,minority_admits,minority_proportion
26509,3345,0.126


total_admits,minority_admits,minority_proportion
6882,1023,0.149


The majority-minority test gap has been the subject of extensive scientific inquiry. Potential causes may include differences in school resources, poverty, family structure, environment, and discrimination.

### Reverse Engineering Current Admissions

If we assume students in our dataset are all students interested in attending law schools, and further assume that [students accepted to top tier law schools all decide to enroll](https://abovethelaw.com/2013/03/which-law-schools-had-the-highest-yield-rate/), we can then reverse engineer an estimated weighting for how law schools make their admissions decisions. We will do this by creating a linear logistic regression model of top tier law school attendance (`TOP_TIER`) based on LSAT, Undergraduate GPA, Minority status, and low income status. We will learn more about this model in a few weeks.

In R, the formulas for statistical models use a special formula object which is created with a common syntax. The formula object is written as "dependent variable" ~ "independent variables" with each indepedent variable seperated with the "+" symbol. 

In [3]:
# fit a logistic regression to predict  
lr_admit <- glm(TOP_TIER ~ LSAT + UGPA + MINORITY + FAM_INC_1, data = data, family="binomial")
summary(lr_admit)


Call:
glm(formula = TOP_TIER ~ LSAT + UGPA + MINORITY + FAM_INC_1, 
    family = "binomial", data = data)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-2.030  -0.765  -0.524   0.799   3.220  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -10.58482    0.17756  -59.61  < 2e-16 ***
LSAT           0.15776    0.00334   47.30  < 2e-16 ***
UGPA           1.04831    0.04035   25.98  < 2e-16 ***
MINORITYTRUE   1.26731    0.04965   25.52  < 2e-16 ***
FAM_INC_1      0.37942    0.10156    3.74  0.00019 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 30361  on 26508  degrees of freedom
Residual deviance: 26224  on 26504  degrees of freedom
AIC: 26234

Number of Fisher Scoring iterations: 4


In the summary table above, column `Estimate` shows the coefficient for each covariate estimated by our logistic regression model.
We can think of the coefficients as different "weights" on different aspects when the admission officer is evaluating an individual applicants.

We can examine our model's accuracy by running:

In [4]:
data <- data %>% 
    mutate(
        # Predicted admittance probability
        lr_pred = predict(lr_admit, data, type="response"),
        # If the probability > 0.5, we will predict the person as being admitted
        lr_pred_admit = lr_pred > 0.5
    )

# Calculate model accuracy, i.e., 78% of our prediction agrees with current admission policy
data %>% 
    summarize(
        prediction_accuracy = mean(TOP_TIER == lr_pred_admit)
    )

prediction_accuracy
0.781


#### Excercise 2: 
Discuss the meaning of this model. What does it say about how law schools are admitting students? How accurate do you think it is? Do you suspect that it is misrepresenting or simplyifing the law school admissions process?

### Simulating Law School Admissions

You will now create an algorithm that simulates law school admissions based on a weighting of your determination. You will create a function that ranks all incoming law students based on their attribures and then admits the top $n$ students to top tier schools, where $n$ is the number of students actually admitted by top tier schools. Specifically in our dataset, $n$ = 6882.

In [9]:
admit_n = 6882

# Sort admission probability in reverse order and take the top admit_n students
sim_admit <- data %>% 
    arrange(desc(lr_pred)) %>%
    slice(1:admit_n)

sim_admit %>%
    summarize(
        total_admits = n(),
        minority_admits = sum(MINORITY),
        minority_proportion = minority_admits/total_admits
    )


total_admits,minority_admits,minority_proportion
6882,1168,0.17


We observe roughly the same minority proportion as in top tier law schools under current admission policy.

#### Excercise 3: Exploring Alternative Admissions Functions

Create a function that simulates an admissions cycle by giving weights to LSAT, Undergraduate GPA, Minority Status, and low-income status. Are you able to create admissions creiteria that match the diverisity of the application pool? Are you able to do so without explictly using race? Recall that _Gratz_ declared using race in a points based way as part of college admissions unconstitutional. 

In [6]:
#WRITE CODE HERE

#BEGIN SOLUTION

my_sim_admit <- data %>% 
    mutate(
        admit_score = LSAT * 1 + UGPA * 5 + FAM_INC_1 * 30 + FAM_INC_2 * 20
    ) %>% 
    arrange(desc(admit_score)) %>%
    slice(1:admit_n)

my_sim_admit %>%
    summarize(
        total_admits = n(),
        minority_admits = sum(MINORITY),
        minority_proportion = minority_admits/total_admits
    )

#END SOLUTION

total_admits,minority_admits,minority_proportion
6882,926,0.135


### Using Predicted Bar Passage as a Selection Criterion

As an additioanl excercise we can consider what law school admissions would look like if law schools selected law students to optimize bar passage rates. This approach might be motivated from two perspectives. First, perhaps using an outcome-based algorithm would allow schools to lessen the weight on LSAT scores, given the critiques of standardized tests as favoring affluent non-minority groups, and hence constitute a "workable race-neutral alternative." Second, more crudely, one of the major inputs into U.S News and World Report law school rankings is bar passage. Schools might want to admit a class to increase bar passages rates or US News might increase the weight of bar passage in its rankings. Our goal here is to examine whether the adoption of such a policy is a workable alternative and whether it might have disparate impact.

#### Excercise 4:


We would now like to simulate admissions based on them admiting individual students who are predicted to pass the bar exam. Create a model to predict bar passage and then use this model to simulate an admissions cycle where the students predicted as being the most likely to pass the bar are admitted into the highest tier law schools. We will create our predictive model using logistic regression.

In [7]:
#Simulate law school admissions based on predicted bar passage

set.seed(330)

# Shuffle data and split train/test set
count <- nrow(data)
train_perc <- .5
train_n = round(count * train_perc)
test_admit_n <- round(admit_n * (1 - train_perc)) # number of admits in the test set

shuffled_data <- data[sample(nrow(data)),]
train <- shuffled_data %>% slice(1:train_n)
test <-shuffled_data %>% slice((train_n+1):count)

# WRITE CODE HERE

# START solution

lm_pass_bar <- glm(PASS_BAR ~ LSAT + UGPA + MALE + MINORITY + FAM_INC, 
                   data = train, family="binomial")

test_admitted <- test %>%
    mutate(
        pass_prob = predict(lm_pass_bar, test, type="response")
    ) %>% 
    arrange(desc(pass_prob)) %>%
    slice(1:test_admit_n)

test_admitted %>%
    summarize(
        total_admits = n(),
        minority_admits = sum(MINORITY),
        minority_proportion = minority_admits/total_admits
    )

#END Solution

total_admits,minority_admits,minority_proportion
3441,61,0.0177


Consider if an admissions office came to you with the proposal of using this model for dertermining which law students that school would admit. How would you evaluate the model and what would you recomemnd to the admissions office? If this model was used, would there be a valid disparate action claim for any applicants who are rejected from top tier schools?

#### Discussion Questions

* One way to characterize the use of bar passage information is as an attempt to reduce the importance of the LSAT in determining law school admissions. Does using bar passage data fulfill the goal of reducing emphasis on the LSAT?

* Consider what some of the potential problems with this data set are. What factors are not represented in the data that might be relevant for predicting outcomes on the bar exam? For success as an attorney? Are their any concerns about state bar passage as an outcome measure? What factors might drive the differences between the different models?  

* How well do these models mimic the procedure of the actual admissions process? How does the performance of actual admissions officers compare to the models we have here and to the extent there are differences in outcomes, what factors might drive those differences? 

* Are there important differences between the populations of interest that may influence the model in undesirable ways? Consider whether minority students are more likely to practice in jurisdictions with lower bar passage rates (e.g., NY or CA)? Consider whether stereotype threat or implicit bias might explain differences in academic or bar passage performance between white and minority students and what implications that has for the approach you've studied above.